## Building Agents with RAG and Tools

### Setting up LLM & Loading the ENV file

In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv(override=True)
load_dotenv('./../.env')

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=400
)

### Setting up PlayWright

In [2]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
import nest_asyncio

nest_asyncio.apply()

In [13]:
async_chrome = create_async_playwright_browser()

toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_chrome)
tools_list = toolkit.get_tools()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Setting up VectorDB & Retriever

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma(
    persist_directory='./../chroma_db_hf', 
    embedding_function=embedding_model
)

result = vector_store.similarity_search("Which country is Parveen travelling ?", k=5)

for doc in result:
    print(doc.metadata["source"])

### Setting up Retriever for the VectorDB

In [15]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k': 5})

# response = retriever.invoke("Full name of Dhanesh?")

# for doc in response:
#     print(doc.metadata["source"])


### Creating a tool to Retrieve from the VectorDB

In [16]:
from langchain.tools import tool

@tool
def retrieve_person_detail(query: str) -> str:
    """
    The tool provides employment details related to DHANESH and PARVEEN where they accepted the job offers.
    You should use this tool to check any employment information related to Dhanesh and Parveen.
    
    Args:
        query: The search query with proper detail
    """
    
    return retriever.invoke(query)

### Addition to the ToolKit

In [17]:
tools_list.append(retrieve_person_detail)

tools_list

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/parveenshaikh/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/par

### Creating Agent code to call the Tools

In [18]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools=tools_list,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Following URL lists some information of DHANESH - https://www.weekday.works/people/dhanesh-gujrathi-dhanesh-gujrathi-9129ab55. Can you check if Dhanesh joined the same organisation in 2019 which applied for his Employment pass that year ?"

response = await agent.ainvoke(query)

response



> Entering new AgentExecutor chain...
<think>
Okay, let's tackle this question. The user wants to know if Dhanesh joined the same organization in 2019 that applied for his Employment pass that year. The provided URL is for Dhanesh's profile on Weekday Works. 

First, I need to visit the URL to get the details. So, I'll use the navigate_browser tool with the given URL. Once there, I should check the profile for any information about his Employment pass or the organization he joined in 2019.

After navigating, I'll use extract_text or get_elements to find relevant sections. Maybe the Employment pass details are listed under a specific section. If there's a mention of the organization's name and the year 2019, that's key. 

If the information isn't there, I might need to look for hyperlinks related to his work history or employment documents. Using extract_hyperlinks could help find relevant pages. However, since the user specifically mentioned the Employment pass, the profile page migh

{'input': 'Following URL lists some information of DHANESH - https://www.weekday.works/people/dhanesh-gujrathi-dhanesh-gujrathi-9129ab55. Can you check if Dhanesh joined the same organisation in 2019 which applied for his Employment pass that year ?',
 'output': 'Based on the information available, Dhanesh Gujrathi was employed by DBS Bank from 2019 to 2020. However, the provided profile does not explicitly mention details about his Employment pass application or the specific organization that applied for it in 2019. To confirm if the organization that applied for his Employment pass in 2019 matches his workplace at that time, additional documentation or clarification would be needed.'}